In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
Folder_A = 'A.The_full_year_MILP'
Folder_B = 'B.Operation_cost'
Folder_D = 'D.Representative_days_based_on_RES_and_Demand'
Folder_E = 'E.Representative_days_based_on_Line_Benefits_OptModel'
Folder_K = 'K.Investments_per_hour'
Folder_L = 'L.Cont_Investments_per_hour'

category_dict = { "FYMILP": Folder_A,
                 "OPC":Folder_B,
                 "R&D": Folder_D ,
                 "OPT_LB": Folder_E,
                 "HI": Folder_K,
                  "CHI": Folder_L,
                }

## Utilization per node Timeseries

In [7]:
def create_utilization_fyts_frame(CaseName_Base,cm,nbc):
    destination_folder = f"Y.FYTS_from_ByStages/{CaseName_Base}/{cm}"

    filename = f"NetworkUtilization_nc{nbc}.csv"


    # Read input data
    df_duration = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Duration_{CaseName_Base}_ByStages_nc{nbc}.csv")
    df_ts_bs = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/4.OutWoInv/oT_Result_NetworkUtilizationPerNode_DC_{CaseName_Base}_ByStages_nc{nbc}.csv")

    # Ensure data types are consistent: convert load levels to strings
    df_duration["LoadLevel"] = df_duration["LoadLevel"].astype(str)
    df_ts_bs["LoadLevel"] = df_ts_bs["LoadLevel"].astype(str)
    df_ts_bs["InitialNode"] = df_ts_bs["InitialNode"].astype(str)
    df_ts_bs["FinalNode"] = df_ts_bs["FinalNode"].astype(str)

    #Filter out line candidates
    df_ts_bs =df_ts_bs[df_ts_bs["Circuit"] == "eac1"]

    # Create mapping of load level to stage name
    load_level_stage_map = df_duration.set_index("LoadLevel")["Stage"].to_dict()
    all_load_levels = df_duration.LoadLevel

    #Create dataframe that that wil hold all timeseries
    frame_values = pd.DataFrame({"LoadLevel":all_load_levels})
    
    #Get unique node pairs connected by an existing line 
    unique_node_pairs = [tuple(x) for x in df_ts_bs[['InitialNode', 'FinalNode']].drop_duplicates().values.tolist()]

    # Create a MultiIndex from the unique node pairs
    multi_index = pd.MultiIndex.from_tuples(unique_node_pairs, names=['InitialNode', 'FinalNode'])

    # Initialize a DataFrame to hold all time series with MultiIndex
    frame_values = pd.DataFrame(index=all_load_levels, columns=multi_index)

    #Select one node pair
    node_pair = unique_node_pairs[0]

    for node_pair in unique_node_pairs:
        print(node_pair)

        initial_node, final_node = node_pair

        # Initialize the full-year time series array
        fy_ts = np.zeros(len(all_load_levels))
        for i, load_level in enumerate(all_load_levels):
            this_loadlevel_stage = load_level_stage_map[load_level]

            # Filter to find the correct reduced load level
            filtered_duration = df_duration[(df_duration["Stage"] == this_loadlevel_stage) & (df_duration["Duration"] == 1)]

            if not filtered_duration.empty:
                reduced_temp_load_level = filtered_duration.LoadLevel.iloc[0]

                # Find the corresponding value in df_ts_bs
                reduced_temp_value = df_ts_bs[(df_ts_bs["InitialNode"] == node_pair[0]) &(df_ts_bs["FinalNode"] == node_pair[1]) & (df_ts_bs["LoadLevel"] == str(reduced_temp_load_level))]

                if not reduced_temp_value.empty:
                    # Assuming you want to assign a value from reduced_temp_value to fy_ts
                    # You might need to aggregate if there are multiple values
                    fy_ts[i] = reduced_temp_value['GWh'].iloc[0]
        frame_values[(initial_node,final_node)] = fy_ts
    return frame_values,destination_folder,filename



In [9]:
#Define case 
CaseName_Base = 'RTS24_mod1'


cm = "CHI"
folder_name = category_dict[cm]

nbcs = [10,20,50,100,200]

for nbc in nbcs:
    frame_values,destination_folder,filename = create_utilization_fyts_frame(CaseName_Base,cm,nbc)
    # Check if the destination folder exists, if not, create it
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    #And write dataframe to destination
    frame_values.to_csv(os.path.join(destination_folder,filename))

('N_120', 'N_123')
('N_113', 'N_123')
('N_119', 'N_120')
('N_108', 'N_110')
('N_112', 'N_123')
('N_104', 'N_109')
('N_112', 'N_113')
('N_121', 'N_122')
('N_108', 'N_109')
('N_114', 'N_116')
('N_101', 'N_103')
('N_117', 'N_118')
('N_107', 'N_108')
('N_111', 'N_114')
('N_116', 'N_117')
('N_105', 'N_110')
('N_103', 'N_124')
('N_109', 'N_111')
('N_117', 'N_122')
('N_102', 'N_106')
('N_103', 'N_109')
('N_118', 'N_121')
('N_111', 'N_113')
('N_109', 'N_112')
('N_110', 'N_111')
('N_110', 'N_112')
('N_101', 'N_105')
('N_115', 'N_116')
('N_101', 'N_102')
('N_102', 'N_104')
('N_106', 'N_110')
('N_115', 'N_124')
('N_115', 'N_121')
('N_116', 'N_119')
('N_112', 'N_113')
('N_115', 'N_124')
('N_110', 'N_112')
('N_103', 'N_109')
('N_113', 'N_123')
('N_106', 'N_110')
('N_120', 'N_123')
('N_104', 'N_109')
('N_118', 'N_121')
('N_111', 'N_113')
('N_119', 'N_120')
('N_105', 'N_110')
('N_101', 'N_105')
('N_107', 'N_108')
('N_110', 'N_111')
('N_116', 'N_119')
('N_114', 'N_116')
('N_117', 'N_122')
('N_101', 'N

## Flow per node 

In [ ]:
def create_flow_fyts_frame(CaseName_Base,cm,nbc):
    destination_folder = f"Y.FYTS_from_ByStages/{CaseName_Base}/{cm}"

    filename = f"Flow_per_node_nc{nbc}.csv"


    # Read input data
    df_duration = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Duration_{CaseName_Base}_ByStages_nc{nbc}.csv")
    df_ts_bs = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/4.OutWoInv/oT_Result_NetworkFlowPerNode_{CaseName_Base}_ByStages_nc{nbc}.csv")

    # Ensure data types are consistent: convert load levels to strings
    df_duration["LoadLevel"] = df_duration["LoadLevel"].astype(str)
    df_ts_bs["LoadLevel"] = df_ts_bs["LoadLevel"].astype(str)
    df_ts_bs["InitialNode"] = df_ts_bs["InitialNode"].astype(str)
    df_ts_bs["FinalNode"] = df_ts_bs["FinalNode"].astype(str)

    #Filter out line candidates
    df_ts_bs =df_ts_bs[df_ts_bs["Circuit"] == "eac1"]

    # Create mapping of load level to stage name
    load_level_stage_map = df_duration.set_index("LoadLevel")["Stage"].to_dict()
    all_load_levels = df_duration.LoadLevel

    #Create dataframe that that wil hold all timeseries
    frame_values = pd.DataFrame({"LoadLevel":all_load_levels})
    
    #Get unique node pairs connected by an existing line 
    unique_node_pairs = [tuple(x) for x in df_ts_bs[['InitialNode', 'FinalNode']].drop_duplicates().values.tolist()]

    # Create a MultiIndex from the unique node pairs
    multi_index = pd.MultiIndex.from_tuples(unique_node_pairs, names=['InitialNode', 'FinalNode'])

    # Initialize a DataFrame to hold all time series with MultiIndex
    frame_values = pd.DataFrame(index=all_load_levels, columns=multi_index)

    #Select one node pair
    node_pair = unique_node_pairs[0]

    for node_pair in unique_node_pairs:
        print(node_pair)

        initial_node, final_node = node_pair

        # Initialize the full-year time series array
        fy_ts = np.zeros(len(all_load_levels))
        for i, load_level in enumerate(all_load_levels[:20]):
            this_loadlevel_stage = load_level_stage_map[load_level]

            # Filter to find the correct reduced load level
            filtered_duration = df_duration[(df_duration["Stage"] == this_loadlevel_stage) & (df_duration["Duration"] == 1)]

            if not filtered_duration.empty:
                reduced_temp_load_level = filtered_duration.LoadLevel.iloc[0]

                # Find the corresponding value in df_ts_bs
                reduced_temp_value = df_ts_bs[(df_ts_bs["InitialNode"] == node_pair[0]) &(df_ts_bs["FinalNode"] == node_pair[1]) & (df_ts_bs["LoadLevel"] == str(reduced_temp_load_level))]

                if not reduced_temp_value.empty:
                    # Assuming you want to assign a value from reduced_temp_value to fy_ts
                    # You might need to aggregate if there are multiple values
                    fy_ts[i] = reduced_temp_value['GWh'].iloc[0]
        frame_values[(initial_node,final_node)] = fy_ts
    return frame_values,destination_folder,filename



In [ ]:
#Define case 
CaseName_Base = 'RTS24_mod1'


cm = "CHI"
folder_name = category_dict[cm]

nbcs = [10,20,50,100,200]

for nbc in nbcs:
    frame_values,destination_folder,filename = create_flow_fyts_frame(CaseName_Base,cm,nbc)
    # Check if the destination folder exists, if not, create it
    if not os.path.exists(destination_folder):
        os.makedirs(destination_folder)
    #And write dataframe to destination
    frame_values.to_csv(os.path.join(destination_folder,filename))

In [3]:
CaseName_Base = 'RTS24_mod1'

cm = "R&D"
folder_name = category_dict[cm]

nbc=20

destination_folder = f"Y.FYTS_from_ByStages/{CaseName_Base}/{cm}"

filename = f"NetworkUtilization_nc{nbc}.csv"


# Read input data
df_duration = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/2.Par/oT_Data_Duration_{CaseName_Base}_ByStages_nc{nbc}.csv")
df_ts_bs = pd.read_csv(f"{folder_name}/{CaseName_Base}_ByStages_nc{nbc}/4.OutWoInv/oT_Result_NetworkUtilizationPerNode_DC_{CaseName_Base}_ByStages_nc{nbc}.csv")

# Ensure data types are consistent: convert load levels to strings
df_duration["LoadLevel"] = df_duration["LoadLevel"].astype(str)
df_ts_bs["LoadLevel"] = df_ts_bs["LoadLevel"].astype(str)
df_ts_bs["InitialNode"] = df_ts_bs["InitialNode"].astype(str)
df_ts_bs["FinalNode"] = df_ts_bs["FinalNode"].astype(str)

#Filter out line candidates
#df_ts_bs =df_ts_bs[df_ts_bs["Circuit"] == "eac1"]

# Create mapping of load level to stage name
load_level_stage_map = df_duration.set_index("LoadLevel")["Stage"].to_dict()
all_load_levels = df_duration.LoadLevel

#Create dataframe that that wil hold all timeseries
frame_values = pd.DataFrame({"LoadLevel":all_load_levels})

# Select a unit
#units = np.unique(df_ts_bs.Unit)

# for unit in units:

#     # Initialize the full-year time series array
#     fy_ts = np.zeros(len(all_load_levels))

#     # Create full-year time series based on mapping and values of representative load levels
#     for i, load_level in enumerate(all_load_levels):
#         this_loadlevel_stage = load_level_stage_map[load_level]

#         # Filter to find the correct reduced load level
#         filtered_duration = df_duration[(df_duration["Stage"] == this_loadlevel_stage) & (df_duration["Duration"] == 1)]

#         if not filtered_duration.empty:
#             reduced_temp_load_level = filtered_duration.LoadLevel.iloc[0]

#             # Find the corresponding value in df_ts_bs
#             reduced_temp_value = df_ts_bs[(df_ts_bs["Unit"] == unit) & (df_ts_bs["LoadLevel"] == str(reduced_temp_load_level))]

#             if not reduced_temp_value.empty:
#                 # Assuming you want to assign a value from reduced_temp_value to fy_ts
#                 # You might need to aggregate if there are multiple values
#                 fy_ts[i] = reduced_temp_value['MW'].iloc[0]  # Replace 'YourValueColumn' with the actual column name
#     frame_values[unit] = fy_ts

In [ ]:
#Get unique node pairs connected by an existing line 
unique_node_pairs = [tuple(x) for x in df_ts_bs[['InitialNode', 'FinalNode']].drop_duplicates().values.tolist()]

# Create a MultiIndex from the unique node pairs
multi_index = pd.MultiIndex.from_tuples(unique_node_pairs, names=['InitialNode', 'FinalNode'])

# Initialize a DataFrame to hold all time series with MultiIndex
frame_values = pd.DataFrame(index=all_load_levels, columns=multi_index)

#Select one node pair
node_pair = unique_node_pairs[0]

for node_pair in unique_node_pairs:
    print(node_pair)

    initial_node, final_node = node_pair

    # Initialize the full-year time series array
    fy_ts = np.zeros(len(all_load_levels))
    for i, load_level in enumerate(all_load_levels):
        this_loadlevel_stage = load_level_stage_map[load_level]

        # Filter to find the correct reduced load level
        filtered_duration = df_duration[(df_duration["Stage"] == this_loadlevel_stage) & (df_duration["Duration"] == 1)]

        if not filtered_duration.empty:
            reduced_temp_load_level = filtered_duration.LoadLevel.iloc[0]

            # Find the corresponding value in df_ts_bs
            reduced_temp_value = df_ts_bs[(df_ts_bs["InitialNode"] == node_pair[0]) &(df_ts_bs["FinalNode"] == node_pair[1]) & (df_ts_bs["LoadLevel"] == str(reduced_temp_load_level))]

            if not reduced_temp_value.empty:
                # Assuming you want to assign a value from reduced_temp_value to fy_ts
                # You might need to aggregate if there are multiple values
                fy_ts[i] = reduced_temp_value['GWh'].iloc[0]  # Replace 'YourValueColumn' with the actual column name
    frame_values[(initial_node,final_node)] = fy_ts


('N_117', 'N_122')
('N_102', 'N_104')
('N_102', 'N_106')
('N_105', 'N_110')
('N_101', 'N_102')
('N_104', 'N_109')
('N_118', 'N_121')
('N_115', 'N_116')
('N_112', 'N_123')
('N_107', 'N_108')
('N_103', 'N_124')
('N_120', 'N_123')
('N_111', 'N_114')
('N_106', 'N_108')
('N_103', 'N_109')
('N_112', 'N_113')
('N_115', 'N_124')
('N_114', 'N_116')
('N_109', 'N_111')
('N_113', 'N_123')
('N_101', 'N_105')


In [15]:
frame_values

InitialNode              N_117     N_102               N_105    N_101   
FinalNode                N_122     N_104     N_106     N_110    N_102   
LoadLevel                                                               
01-01 00:00:00+01:00  0.002298  0.012218  0.276988  0.235467  0.66507  \
01-01 01:00:00+01:00  0.002298  0.012218  0.276988  0.235467  0.66507   
01-01 02:00:00+01:00  0.002298  0.012218  0.276988  0.235467  0.66507   
01-01 03:00:00+01:00  0.002298  0.012218  0.276988  0.235467  0.66507   
01-01 04:00:00+01:00  0.002298  0.012218  0.276988  0.235467  0.66507   
...                        ...       ...       ...       ...      ...   
12-30 19:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.00000   
12-30 20:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.00000   
12-30 21:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.00000   
12-30 22:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.00000   
12-30 23:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.00000   

InitialNode              N_104 N_118 N_115    N_112     N_107  ...     N_110   
FinalNode                N_109 N_121 N_116    N_123     N_108  ...     N_112   
LoadLevel                                                      ...             
01-01 00:00:00+01:00  0.487303   0.0   0.0  0.08008  0.178455  ...  0.109616  \
01-01 01:00:00+01:00  0.487303   0.0   0.0  0.08008  0.178455  ...  0.109616   
01-01 02:00:00+01:00  0.487303   0.0   0.0  0.08008  0.178455  ...  0.109616   
01-01 03:00:00+01:00  0.487303   0.0   0.0  0.08008  0.178455  ...  0.109616   
01-01 04:00:00+01:00  0.487303   0.0   0.0  0.08008  0.178455  ...  0.109616   
...                        ...   ...   ...      ...       ...  ...       ...   
12-30 19:00:00+01:00  0.000000   0.0   0.0  0.00000  0.000000  ...  0.000000   
12-30 20:00:00+01:00  0.000000   0.0   0.0  0.00000  0.000000  ...  0.000000   
12-30 21:00:00+01:00  0.000000   0.0   0.0  0.00000  0.000000  ...  0.000000   
12-30 22:00:00+01:00  0.000000   0.0   0.0  0.00000  0.000000  ...  0.000000   
12-30 23:00:00+01:00  0.000000   0.0   0.0  0.00000  0.000000  ...  0.000000   

InitialNode              N_119     N_121     N_101     N_106     N_108   
FinalNode                N_120     N_122     N_103     N_110     N_109   
LoadLevel                                                                
01-01 00:00:00+01:00  0.051525  0.002298  0.146579  0.079294  0.248013  \
01-01 01:00:00+01:00  0.051525  0.002298  0.146579  0.079294  0.248013   
01-01 02:00:00+01:00  0.051525  0.002298  0.146579  0.079294  0.248013   
01-01 03:00:00+01:00  0.051525  0.002298  0.146579  0.079294  0.248013   
01-01 04:00:00+01:00  0.051525  0.002298  0.146579  0.079294  0.248013   
...                        ...       ...       ...       ...       ...   
12-30 19:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.000000   
12-30 20:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.000000   
12-30 21:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.000000   
12-30 22:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.000000   
12-30 23:00:00+01:00  0.000000  0.000000  0.000000  0.000000  0.000000   

InitialNode                       N_110     N_116     N_115  
FinalNode               N_110     N_111     N_119     N_121  
LoadLevel                                                    
01-01 00:00:00+01:00  0.19354  0.076084  0.097851  0.581913  
01-01 01:00:00+01:00  0.19354  0.076084  0.097851  0.581913  
01-01 02:00:00+01:00  0.19354  0.076084  0.097851  0.581913  
01-01 03:00:00+01:00  0.19354  0.076084  0.097851  0.581913  
01-01 04:00:00+01:00  0.19354  0.076084  0.097851  0.581913  
...                       ...       ...       ...       ...  
12-30 19:00:00+01:00  0.00000  0.000000  0.000000  0.000000  
12-30 20:00:00+01:00  0.00000  0.000000  0.000000  0.000000  
12-30 21:00:00+01:00  0.00000  0.000000  0.000000  0.000000  
12-30 22:00:00+01:00  0.00000  0.000000  0.000000  0.000000  
12-30 23:00:0

In [6]:
frame_values

,LoadLevel,N_101-N_102
0,01-01 00:00:00+01:00,0.66507
1,01-01 01:00:00+01:00,0.66507
2,01-01 02:00:00+01:00,0.66507
3,01-01 03:00:00+01:00,0.66507
4,01-01 04:00:00+01:00,0.66507
...,...,...
8731,12-30 19:00:00+01:00,0.00000
8732,12-30 20:00:00+01:00,0.00000
8733,12-30 21:00:00+01:00,0.00000
8734,12-30 22:00:00+01:00,0.00000


In [ ]:
# Extract unique node pairs from df_ts_bs
unique_node_pairs = df_ts_bs[['InitialNode', 'FinalNode']].drop_duplicates().values

# Initialize a DataFrame to hold all time series
frame_values = pd.DataFrame({"LoadLevel": all_load_levels})

# Iterate over each unique node pair
for node_pair in unique_node_pairs:
    initial_node, final_node = node_pair

    # Initialize the full-year time series array for the current node pair
    fy_ts = np.zeros(len(all_load_levels))
    
    for i, load_level in enumerate(all_load_levels):
        this_loadlevel_stage = load_level_stage_map[load_level]

        # Filter to find the correct reduced load level
        filtered_duration = df_duration[
            (df_duration["Stage"] == this_loadlevel_stage) & 
            (df_duration["Duration"] == 1)
        ]

        if not filtered_duration.empty:
            reduced_temp_load_level = filtered_duration.LoadLevel.iloc[0]

            # Find the corresponding value in df_ts_bs
            reduced_temp_value = df_ts_bs[
                (df_ts_bs["InitialNode"] == initial_node) &
                (df_ts_bs["FinalNode"] == final_node) &
                (df_ts_bs["LoadLevel"] == str(reduced_temp_load_level))
            ]

            if not reduced_temp_value.empty:
                # Assign the value to the time series array
                fy_ts[i] = reduced_temp_value['GWh'].iloc[0]

    # Add the time series for the current node pair to the DataFrame
    frame_values[f"{initial_node}-{final_node}"] = fy_ts

# Print the first node pair's time series as an example
first_node_pair = unique_node_pairs[0]
first_node_pair_key = f"{first_node_pair[0]}-{first_node_pair[1]}"
print(f"Time series for node pair {first_node_pair_key}:")
print(frame_values[first_node_pair_key])


In [47]:
fy_ts[15]

0.0740160494921053

In [13]:
load_level = all_load_levels[0]
load_level

'01-01 00:00:00+01:00'

In [7]:
df_ts_bs.GWh.sum()

82.92351985661828

In [8]:
df_ts_bs[df_ts_bs["Circuit"] == "eac1"].GWh.sum()

82.92351985661828